In [ ]:
#imports
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
import pandas as pd
import kagglehub

In [ ]:
path = kagglehub.dataset_download("rmisra/news-category-dataset")
df=pd.read_json(path, lines=True)
#df=pd.read_json("C:\\Users\\Gargi Joshi\\Build-your-own-GPT-bot\\dataset\\News_Category_Dataset_v3.json", lines=True)
headlines= df['headline']

documents=[]
for i in range(1000): #entire data is 205927 or smtg nd i was done with the loading tf
    documents.append(headlines[i])

In [ ]:
snowballstemmer=SnowballStemmer('english')
all_stopwords=stopwords.words('english')

In [ ]:
words = [word_tokenize(doc.lower()) for doc in documents]

for i in range(len(words)):
    words[i]=[snowballstemmer.stem(word) for word in words[i] if word not in all_stopwords]

In [ ]:
model=Word2Vec(words, min_count=1)
vocabulary = list(model.wv.index_to_key)
avg_vector=[]
for i in range(len(words)-1):
    sum_vectors=0
    for j in range(len(words[i])):
        sum_vectors=sum_vectors+model.wv[words[i][j]]
    avg_vector.append(sum_vectors/len(words[i]))

In [ ]:
input_sent=str(input("enter sentence:"))
input_words=word_tokenize(input_sent.lower())
for i in range(len(input_words)):
    input_words[i]=snowballstemmer.stem(input_words[i])
input_vector=[model.wv[word] for word in input_words if word in vocabulary]
sum_input=0
for i in range(len(input_vector)):
    sum_input=sum_input+input_vector[i]
avg_input_vector=sum_input/len(input_vector)


In [ ]:
import numpy as np

cosine_similarity= [(np.dot(avg_input_vector, avg_i)/(np.linalg.norm(avg_input_vector)* np.linalg.norm(avg_i))) for avg_i in avg_vector]
distance=[1-cosine for cosine in cosine_similarity]
ascending_order=sorted(distance)

In [ ]:
order=[]
for i in range(5):
    for j in range(len(ascending_order)):
        if ascending_order[i]==distance[j]:
            order.append(j)
            break


In [ ]:
top_5=[documents[order[i]] for i in range(5)]
print(top_5)